In [1]:
import numpy as np
import pandas as pd

from funcs.model import FlareModulator

import astropy.units as u
from astropy.constants import R_sun

torad = lambda x: x / 180. * np.pi

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
NumExpr defaulting to 8 threads.


In [16]:
# Get best-fit results
# _deprecated
df = pd.read_csv("../results/mcmc/15_12_2020_GP_deprecated_mcmcoutput.csv")

# Pick up stellar parameters
props = pd.read_csv(f"../../data/summary/lcs.csv")

# Get MCMC inputs
mcmcinit = pd.read_csv(f"../../data/summary/inits_decoupled_GP.csv")

In [26]:
row = df.iloc[6]
row.suffix

'c'

# 1-flare solution

In [27]:
# get the right suffix
# if np.isnan(row.suffix):
#     row.suffix = ""
    
# row.suffix="c"

# Pick your star
target = props[props.ID == int(row.ID)].iloc[0]

# get qlum and R as they were passed to MCMC
inits = mcmcinit[(mcmcinit.ID.astype(str) == str(row.ID) + str(row.suffix)) &
                 (mcmcinit.tstamp == row.tstamp)].iloc[0]

qlum = inits.qlum_erg_s * u.erg / u.s
R = inits.R_Rsun * R_sun

# Get light curve
lc = pd.read_csv(f"../../data/lcs/{row.tstamp}_{row.ID}.csv")

# Get flux median
median = lc.median_[0]

# Convert MCMC outputs to model inputs

# Get peak time in phase units
phi_a = (row.t0_d_50 -lc.t[0]) / target.Prot_d * 2 * np.pi 

# Convert flare peak phase to phase at t0
phi0 = phi_a % (2. * np.pi) - (row.phase_deg_50  / 180 * np.pi)

# Get FHWM values in phase units
fwhm1 = row.fwhm1_d_50 / target.Prot_d * 2 * np.pi
fwhm2 = row.fwhm2_d_50 / target.Prot_d * 2 * np.pi

# Init FlareModulator object
F =  FlareModulator(lc.phi.values, lc.flux.values, lc.flux_err.values, qlum=qlum, R=R,
                 median=median, nflares=1, iscoupled=False)

# Assemble best-fit parameters and constants
params = [torad(row.latitude_deg_50),
          phi0,
          torad(row.i_deg_50),
          row.a_50, 
          phi_a, 
          fwhm1, 
          fwhm2]

# Get log-likelihood for best-fiit solution
logL = F.log_likelihood(params) # the closer to 0 the better, always negative

# Calculate Chi^2 value
chi2 = F.chi_square(params)

# Calculate Bayesian information criterion
# second term is always positive, 
# light curve length stays constant across different fits
# number of parameters varies with 1- vs. 2-flare solution
BIC = len(params) * np.log(len(lc.flux)) - 2 * logL 

In [28]:
print(f"Chi^2 for TIC {row.ID}, fit with 1 flare fit: {chi2} ({row.tstamp})")

Chi^2 for TIC 44984200, fit with 1 flare fit: 144.13400509979132 (10_12_2020_07_12)


# 2-flare solution

In [29]:
df.suffix

0    NaN
1      a
2      b
3      a
4      b
5    NaN
6      c
Name: suffix, dtype: object

In [33]:
# get the right suffix
# if np.isnan(row.suffix):
#     row.suffix = ""
fp = []

for j, row in df.iloc[3:5].iterrows():

    # Pick your star
    target = props[props.ID == int(row.ID)].iloc[0]

    # get qlum and R as they were passed to MCMC
    inits = mcmcinit[(mcmcinit.ID.astype(str) == str(row.ID) + str(row.suffix)) &
                     (mcmcinit.tstamp == row.tstamp)].iloc[0]

    qlum = inits.qlum_erg_s * u.erg / u.s
    R = inits.R_Rsun * R_sun

    # Get light curve
    lc = pd.read_csv(f"../../data/lcs/{row.tstamp}_{row.ID}.csv")

    # Get flux median
    median = lc.median_[0]


    # Init FlareModulator object
    F =  FlareModulator(lc.phi.values, lc.flux.values, lc.flux_err.values, qlum=qlum, R=R,
                     median=median, nflares=2, iscoupled=False)

    # Convert MCMC outputs to model inputs

    # Get peak time in phase units
    phi_a = (row.t0_d_50 -lc.t[0]) / target.Prot_d * 2 * np.pi 

    # Convert flare peak phase to phase at t0
    phi0 = phi_a % (2. * np.pi) - (row.phase_deg_50  / 180 * np.pi)

    # Get FHWM values in phase units
    fwhm1 = row.fwhm1_d_50 / target.Prot_d * 2 * np.pi
    fwhm2 = row.fwhm2_d_50 / target.Prot_d * 2 * np.pi
    
    fp.append(row.a_50)
    fp.append(phi_a)
    fp.append(fwhm1)
    fp.append(fwhm2)


# Assemble best-fit parameters and constants
params = [torad(row.latitude_deg_50),
          phi0,
          torad(row.i_deg_50)]

params =  params + fp
params

[1.4967875097171692,
 0.2144705873240973,
 0.5774591887541942,
 0.319417523017479,
 12.167705466465494,
 1.868707456665623,
 3.1107704584324205,
 0.06877716336343401,
 17.78922934075829,
 1.655707171595105,
 0.6212968235149064]

In [34]:
# Get log-likelihood for best-fiit solution
logL = F.log_likelihood(params) # the closer to 0 the better, always negative

# Calculate Chi^2 value
chi2 = F.chi_square(params)


# Calculate Bayesian information criterion
# second term is always positive, 
# light curve length stays constant across different fits
# number of parameters varies with 1- vs. 2-flare solution
BIC = len(params) * np.log(len(lc.flux)) - 2 * logL 

In [35]:
print(f"BIC for TIC {row.ID}, fit with 2 flare fit: {BIC} ({row.tstamp})")
print(f"Chi^2 for TIC {row.ID}, fit with 2 flare fit: {chi2} ({row.tstamp})")

BIC for TIC 44984200, fit with 2 flare fit: 6741.812040032524 (07_12_2020_19_42)
Chi^2 for TIC 44984200, fit with 2 flare fit: 107.3520562886087 (07_12_2020_19_42)


### TIC 452

**final**: BIC for TIC 452922110, fit with 1 flare fit: 3383.8764766286595
final: Chi^2 for TIC 452922110, fit with 1 flare fit: 114.56119373881997 (10_12_2020_07_05)

deprecrated: BIC for TIC 452922110, fit with 1 flare fit: 3539.4625885910755
**deprecated**: Chi^2 for TIC 452922110, fit with 1 flare fit: 105.50394277965948 (07_12_2020_15_44)

### TIC 277

final: BIC for TIC 277539431, fit with 1 flare fit: 5975.642718484921
final: Chi^2 for TIC 277539431, fit with 1 flare fit: 85.35280228125123 (07_12_2020_15_26)


### TIC 449

deprecated: BIC for TIC 44984200 (normal_uncertainty), fit with 1 flare fit: 6239.3862509798755
deprecated: Chi^2 for TIC 44984200 (normal_uncertainty), fit with 1 flare fit: 147.57585678156408 (07_12_2020_18_46)

**deprecated**: BIC for TIC 44984200 (c, high_uncertainties_peak1), fit with 1 flare fit: 4484.622756262301
deprecated: Chi^2 for TIC 44984200, fit with 1 flare fit: 144.13400509979132 (10_12_2020_07_12)

deprecated: BIC for TIC 44984200, fit with 2 flare fit: 6741.812040032524 (07_12_2020_19_42)
**deprecated**: Chi^2 for TIC 44984200, fit with 2 flare fit: 107.3520562886087 (07_12_2020_19_42)

BIC for TIC 44984200, fit with 2 flare fit: 5781.215417165257 (10_12_2020_07_10)
Chi^2 for TIC 44984200, fit with 2 flare fit: 255.0029659518811 (10_12_2020_07_10)


### TIC 237

**final**: BIC for TIC 237880881, fit with 2 flare fit: 17316.74773510112 (10_12_2020_07_11)
**final**: Chi^2 for TIC 237880881, fit with 2 flare fit: 1523.8682823327727 (10_12_2020_07_11)

deprecated: BIC for TIC 237880881, fit with 2 flare fit: 97020.40734765412  (07_12_2020_07_47)
deprecated: Chi^2 for TIC 237880881, fit with 2 flare fit: 3337.264890932395 (07_12_2020_07_47)


In [ ]:
0.0044 / 0.1446

In [42]:
0.0084 / 0.2746

0.030589949016751636

##### 